# 🎭 Wariacyjny Autoenkoder do Uczenia Reprezentacji Twarzy (CelebA)


## Cele

- Opracowanie i implementacja architektury VAE w **TensorFlow**.  
- Nauka znaczących reprezentacji latentnych, które opisują cechy twarzy.  
- Generacja i rekonstrukcja obrazów twarzy z wykorzystaniem przestrzeni latentnej.  
- Wizualizacja interpolacji i manipulacji atrybutami w przestrzeni latentnej.


## Dataset

CelebA to duży zbiór obrazów twarzy celebrytów, powszechnie wykorzystywany w badaniach nad rozpoznawaniem i generowaniem cech twarzy. Jego główne atuty to duża liczba próbek, bogate adnotacje atrybutów oraz zróżnicowane warunki (pozycje głowy, tło), co czyni go idealnym do trenowania głębokich modeli generatywnych, takich jak VAE. Na potrzeby tego projektu wykorzystywane będą dane opublikowane na platformie Kaggle ([link](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset))

### Dane wejściowe

- obrazy RGB zapisane w formacie .jpg o wymiarach 178 x 218 pikseli
- 40 binarnych atrybutów (np. Smiling, Wearing_Hat)
- Obrazy są wstępnie wyrównane i przycięte, zawierają celebrytów w różnych pozach i warunkach oświetleniowych

### Dane wyjściowe

W przypadku kodera:
- reprezentacja obrazu w przestrzeni latentnej
W przypadku dekodera: 
- zrekonstruowane obrazy RGB o wymiarach 178 x 128 pikseli
- wygenerowane nowe obrazy z kontrolowanymi zmianami cech

## Architektura VAE

Wariacyjny Autoenkoder składa się z kodera, który zamienia obraz na wektor latentny oraz dekodera, który rekonstruuje obraz z tego wektora, ucząc się minimalizować błąd rekonstrukcji i odchylenie KL pomiędzy rozkładami. Każda z części modelu będzie zbudowana z wykorzystaniem warstw konwolucyjnych.

### Koder
- kilka bloków warstw konwolucyjnych 2D (Conv2D → BatchNorm → ReLU) ze stopniowym zmniejszaniem liczby filtrów
- Flatten i warstwy w pełni połączone
- wyjście w postaci 2 wektorów: wektora średnich ($\mu$) i wektora log wariacji (log($\sigma^2$))o długościach zgodnych z wymiarem przestrzeni latentnej

### Dekoder
- warstwy gęste + reshaping
- seria bloków analogiczna do kodera (Conv2DTranspose lub UpSampling2D + Conv2D)

### Funkcja straty
Funkcja straty będzie kombinacją liniową dwóch elementów:
- strata rekonstrukcji (np. MSE między oryginalnym a odtworzonym zdjęciem)
- KL Divergence - regularyzacja rozkładu latentnego w kierunku rozkładu normalnego

Ważnym elementem tego typu modeli jest tzw. sztuczka reparametryzacji:
Aby umożliwić propagację gradientów, wprowadzamy deterministyczną transformację ($z$ - wektor z przestrzeni latentnej):​

$z = \mu + \sigma\cdot\epsilon,\quad\epsilon\sim N(0,1)$

W tym podejściu losowość jest izolowana w zmiennej $\epsilon$, która jest niezależna od parametrów modelu. Dzięki temu operacja próbkowania staje się deterministyczna względem $\mu$ i $\sigma$, co pozwala na efektywne obliczanie gradientów i trenowanie modelu metodami gradientowymi.​